In [11]:
import dropbox
import os
import numpy as np
import pandas as pd
from scipy.stats import t
import math
import collections
from PIL import Image
from dropbox import DropboxOAuth2FlowNoRedirect
import matplotlib.pyplot as plt
import cv2
import pypyodbc
from sklearn.preprocessing import LabelEncoder

from keras.applications.xception import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import concatenate, Input, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical  #?

In [2]:
cnxn = pypyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=028-GUMUS1-SRV;"
                      "Database=KOTON_DB;"
                      "Trusted_Connection=yes;")

In [3]:
access_token='0DQLcsXIojcAAAAAAAgybeB3DOmXKK7bRTLyYwkthbrAiGmpQR4AuGINQkjBXhif'
dbx = dropbox.Dropbox(access_token)

In [4]:
retrieved_image_df = pd.read_excel('C:/Users/Recep/Koton/Data/retrieved_image_list.xlsx', sheet_name='Sheet1')

In [5]:
def data_set(codes, indexes):
    """Takes list of option codes as input and returns related dataset, response set (sales amount)
    and genuine option codes that included in e-commerce table.
    """
    
    if len(codes)>2000:
        numOfBatches = 100
    else:
        numOfBatches = 1
    
    string_tar = "Select OptionCode, TTSales = ISNULL(sum(TotalSales), 0) \
    from (Select CODE, TotalSales \
    From ( \
    SELECT [CODE], TotalSales=Sum(isnull(cast(totalqty as float),0)) \
      FROM [KOTON_DB].[dbo].[FA_Sales_Ecommerce] Group by CODE)t) AS Sales INNER JOIN [KOTON_DB].[dbo].[LU_Variant_New] AS Variant ON Variant.SkuCode = Sales.Code \
    Where "
    
    codes_split = np.array_split(codes, numOfBatches)
    indexes_split = np.array_split(indexes, numOfBatches)
    
    # to prevent empty targets
    gen_codes = []
    gen_indexes = []
    target = []
    
    for batch in range(numOfBatches):
        _ =[]
        flag2 = True
        for i in codes_split[batch]:
            if flag2:
                _.append("OptionCode='"+i+"'")
                flag2 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query2 = string_tar + ''.join(_) + " Group by OptionCode"
        temp = pd.read_sql_query(query2, cnxn, index_col='optioncode')
        target.append(temp)
    target = pd.concat(target)

    for code, index in zip(codes, indexes):
        try:
            target.loc[code]
            gen_codes.append(code)
            gen_indexes.append(index)
        except:
            pass        
        
    
    string_inp = "SELECT DISTINCT OptionCode, ProductHierarchyLevel02, ProductHierarchyLevel03, ProductHierarchyLevel04, ProductHierarchyLevel05, SeasonCode, KOLEKSİYON, [KULLANIM-eski], [MODEL TASARIMI], [OZEL KOLEKSIYON], SEZONU \
  FROM [KOTON_DB].[dbo].[LU_Model_New] AS Model \
  INNER JOIN [KOTON_DB].[dbo].[LU_Variant_New] AS Variant ON Variant.ItemCode = Model.ItemCode \
  WHERE "
    
#     numOfBatches = max(min(len(gen_indexes), numOfBatches), 1)
    numOfBatches = min(len(gen_indexes), numOfBatches)
    gen_codes_splited = np.array_split(gen_codes, numOfBatches)
    dataset = []
    
    for batch in range(numOfBatches):
        _ =[]
        flag1 = True
        for i in gen_codes_splited[batch]:
            if flag1:
                _.append("OptionCode='"+i+"'")
                flag1 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query1 = string_inp + ''.join(_)
        dataset.append(pd.read_sql_query(query1, cnxn, index_col='optioncode'))
    
    dataset = pd.concat(dataset)
    
    
    return dataset, target, gen_codes, gen_indexes

In [6]:
number_of_samples = 50
startIndex = 16000

In [13]:
def getImages_n_train(numberSamples, startIndex, batchSize=64):
    errorLog = []
    bn_model = Xception(include_top=False, weights='imagenet')
    
    first_input = Input(shape=(10, 10, 2048))
    first = Flatten()(first_input)
    
    second_input = Input(shape=(10,)) # number of features=10 !!!Change it after word2vec
    second = Dense(10, activation='relu')(second_input)
    
    model = concatenate([first, second], axis=1)
    model = Dense(256, activation='relu')(model)
    model = Dropout(0.5)(model)
    model = Dense(256, activation='relu')(model)
    model = Dropout(0.5)(model)
    model = Dense(1, activation='relu')(model)
    
    model_ = Model(inputs=[first_input, second_input], outputs=model)

    model_.compile(optimizer='adam',
                  loss='mean_squared_error', metrics=['mae'])
    
    codes = []
    idx = []
    for picture_index in range(startIndex,startIndex + numberSamples):
        code = retrieved_image_df.iloc[picture_index].name
        codes.append(code)
        idx.append(picture_index)
    
    dset, tar, gen_codes, gen_indexes = data_set(codes, idx)
    dset = dset.apply(LabelEncoder().fit_transform)
#     dset = pd.get_dummies(dset)
    print(dset.shape)
        
    if batchSize>len(dset):
        batchSize = len(dset)
        print('Batch size is automatically adjusted.')
    
    numOfBatches, lastBatch = divmod(len(dset), batchSize-1)
    dsets = np.array_split(dset, numOfBatches)
    tars = np.array_split(tar, numOfBatches)
    gen_codes = np.array_split(gen_codes, numOfBatches)
    gen_indexes = np.array_split(gen_indexes, numOfBatches)
    
    for batch in range(numOfBatches):
        pic1 =[]
        print('Batch: '+str(batch+1)+'/'+str(numOfBatches))
        for code, picture_index in zip(gen_codes[batch], gen_indexes[batch]):
            im_index = retrieved_image_df.iloc[picture_index]['image1'+'_index']        
            if (math.isnan(float(im_index))==False):
                filename = str(code).lower()+'_'+'image1'+'_'+str(int(im_index))+'.jpg'
                path = '/Koton_Image_Files/'+filename
                try:
                    md, res = dbx.files_download(path)
                    temp_pic = cv2.resize(cv2.imdecode(np.frombuffer(res.content, dtype=np.uint8), -1), (299,299))
                    assert temp_pic.shape == (299, 299, 3)
                    pic1.append(temp_pic)
                except:
                    errorLog.append(str(im_index)+'NAN value')
                    batch += 1 #ignore defective batch
                    continue
            else:
                errorLog.append(path+' Not found')
                batch += 1 #ignore defective batch
                continue
        pic1 = np.array(pic1)
        bottleneck_features = bn_model.predict(pic1)
        model_.fit([bottleneck_features, dsets[batch]], tars[batch],
              epochs=10)
      
    return model_, errorLog
                    


In [14]:
model, errorLog = getImages_n_train(900, 1724, 64)

(41, 10)
Batch size is automatically adjusted.
Batch: 1/1
Epoch 1/10
41/41 [==============================] - 5s 129ms/step - loss: 9196.5419 - mean_absolute_error: 68.6840
Epoch 2/10
41/41 [==============================] - 2s 47ms/step - loss: 76.4390 - mean_absolute_error: 3.3659
Epoch 3/10
41/41 [==============================] - 2s 48ms/step - loss: 76.4390 - mean_absolute_error: 3.3659
Epoch 4/10
41/41 [==============================] - 2s 45ms/step - loss: 76.4390 - mean_absolute_error: 3.3659
Epoch 5/10
41/41 [==============================] - 2s 48ms/step - loss: 76.4390 - mean_absolute_error: 3.3659
Epoch 6/10
32/41 [======================>.......] - ETA: 0s - loss: 9.1562 - mean_absolute_error: 1.6562

KeyboardInterrupt: 